In [12]:
# 02_quality_coverage.ipynb
# Purpose: Assess data quality and coverage of the cleaned Ames Housing dataset.

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# Optional: nicer tables in notebook
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

In [13]:
print("=== Loading cleaned dataset ===")

clean_path = "data/cleaned/ames_cleaned.csv"

df_clean = pd.read_csv(
    clean_path,
    keep_default_na=False,   # NÃO converter strings como 'None' em NaN
    na_values=[]             # não adicionar outros marcadores de NA
)

print("Loaded dataframe with shape:", df_clean.shape)
print("Total missing values:", df_clean.isna().sum().sum())
df_clean.head()

=== Loading cleaned dataset ===
Loaded dataframe with shape: (2930, 81)
Total missing values: 0


,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,526301100,20,RL,141.0,31770,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960-01-01,1960-01-01,Hip,CompShg,BrkFace,Plywood,Stone,112.0,3,3,CBlock,3,4,4,4,639.0,1,0.0,441.0,1080.0,GasA,2,Y,SBrkr,1656,0,0,1656,1,0,1,0,3,1,3,7,8,2,4,Attchd,1960-01-01,Fin,2,528.0,3,3,P,210,62,0,0,0,0,0,0,None,0,5,2010-01-01,WD,Normal,215000
1,526350040,20,RH,80.0,11622,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961-01-01,1961-01-01,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,3,CBlock,3,3,1,3,468.0,2,144.0,270.0,882.0,GasA,3,Y,SBrkr,896,0,0,896,0,0,1,0,2,1,3,5,8,0,0,Attchd,1961-01-01,Unf,1,730.0,3,3,Y,140,0,0,0,120,0,0,0,None,0,6,2010-01-01,WD,Normal,105000
2,526351010,20,RL,81.0,14267,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958-01-01,1958-01-01,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,3,3,CBlock,3,3,1,5,923.0,1,0.0,406.0,1329.0,GasA,3,Y,SBrkr,1329,0,0,1329,0,0,1,1,3,1,4,6,8,0,0,Attchd,1958-01-01,Unf,1,312.0,3,3,Y,393,36,0,0,0,0,0,0,Gar2,12500,6,2010-01-01,WD,Normal,172000
3,526353030,20,RL,93.0,11160,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968-01-01,1968-01-01,Hip,CompShg,BrkFace,BrkFace,None,0.0,4,3,CBlock,3,3,1,5,1065.0,1,0.0,1045.0,2110.0,GasA,5,Y,SBrkr,2110,0,0,2110,1,0,2,1,3,1,5,8,8,2,3,Attchd,1968-01-01,Fin,2,522.0,3,3,Y,0,0,0,0,0,0,0,0,None,0,4,2010-01-01,WD,Normal,244000
4,527105010,60,RL,74.0,13830,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997-01-01,1998-01-01,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,3,PConc,4,3,1,6,791.0,1,0.0,137.0,928.0,GasA,4,Y,SBrkr,928,701,0,1629,0,0,2,1,3,1,3,6,8,1,3,Attchd,1997-01-01,Fin,2,482.0,3,3,Y,212,34,0,0,0,0,0,0,None,0,3,2010-01-01,WD,Normal,189900


In [14]:
# === Step 1: Missingness & Coverage Assessment ===
print("=== Missingness & Coverage Assessment ===\n")

missing_pct = (df_clean.isna().sum() / len(df_clean) * 100).sort_values(ascending=False)

print("[1] Percentage of missing values per column (should be 0% after cleaning):\n")
print(missing_pct)

# Quick check: total number of missing entries in entire dataset
total_missing = df_clean.isna().sum().sum()
print(f"\nTotal missing values in df_clean: {total_missing}")

=== Missingness & Coverage Assessment ===

[1] Percentage of missing values per column (should be 0% after cleaning):

PID              0.0
Central Air      0.0
Garage Yr Blt    0.0
Garage Type      0.0
Fireplace Qu     0.0
                ... 
Mas Vnr Area     0.0
Mas Vnr Type     0.0
Exterior 2nd     0.0
Exterior 1st     0.0
SalePrice        0.0
Length: 81, dtype: float64

Total missing values in df_clean: 0


In [15]:
# === Step 2: Low-Variability Variable Detection ===
print("\n=== Low-Variability Check ===\n")

# Count unique values per column
unique_counts = df_clean.nunique().sort_values()

print("[2] Number of unique values per column (ascending):\n")
print(unique_counts)

# Identify quasi-constant variables (threshold can be tuned)
threshold = 2  # meaning: only 1 or 2 unique values
low_variability_cols = unique_counts[unique_counts <= threshold].index.tolist()

print(f"\nColumns with very low variability (<= {threshold} unique values):")
print(low_variability_cols)


=== Low-Variability Check ===

[2] Number of unique values per column (ascending):

Fence                1
Central Air          2
Street               2
Bsmt Half Bath       3
Half Bath            3
                  ... 
1st Flr SF        1083
Bsmt Unf SF       1137
Gr Liv Area       1292
Lot Area          1960
PID               2930
Length: 81, dtype: int64

Columns with very low variability (<= 2 unique values):
['Fence', 'Central Air', 'Street']


In [16]:
# === Step 3: Outlier Detection (IQR Method) ===
print("\n=== Outlier Detection (IQR Method) ===\n")

# Select numeric columns only
numeric_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns

outlier_summary = {}

for col in numeric_cols:
    q1 = df_clean[col].quantile(0.25)
    q3 = df_clean[col].quantile(0.75)
    iqr = q3 - q1

    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    outliers = ((df_clean[col] < lower) | (df_clean[col] > upper)).sum()
    outlier_summary[col] = outliers

# Convert to a sorted Series for readability
outlier_summary = pd.Series(outlier_summary).sort_values(ascending=False)

print("[3] Number of outliers per numeric variable (sorted):\n")
print(outlier_summary.head(20))   # top 20 most problematic columns

print("\nVariables with no outliers:")
print(outlier_summary[outlier_summary == 0].index.tolist())


=== Outlier Detection (IQR Method) ===

[3] Number of outliers per numeric variable (sorted):

Enclosed Porch    459
BsmtFin Type 2    431
Exter Cond        381
BsmtFin SF 2      351
Garage Qual       315
Bsmt Cond         314
Bsmt Exposure     284
Garage Cond       265
Screen Porch      256
Overall Cond      252
MS SubClass       208
Lot Frontage      206
Mas Vnr Area      203
Functional        202
Bsmt Half Bath    175
Open Porch SF     159
SalePrice         137
Kitchen AbvGr     134
Lot Area          127
Total Bsmt SF     123
dtype: int64

Variables with no outliers:
['Fence', 'Mo Sold', 'Fireplace Qu', 'Exter Qual', 'BsmtFin Type 1', 'Heating QC', 'Half Bath', 'PID']


In [17]:
# === Step 4: Category Coverage & Dominance ===
print("\n=== Category Coverage & Dominance ===\n")

categorical_cols = df_clean.select_dtypes(include=["object", "string"]).columns

for col in categorical_cols:
    print(f"\n--- {col} ---")
    value_counts = df_clean[col].value_counts(normalize=True).round(3)
    print(value_counts)

    # Identify dominant categories
    top_category = value_counts.index[0]
    top_pct = value_counts.iloc[0]
    
    if top_pct > 0.85:
        print(f"!! Warning: '{col}' is dominated by category '{top_category}' ({top_pct*100:.1f}% of rows).")
    
    # Identify rare categories
    rare_cats = value_counts[value_counts < 0.01].index.tolist()
    if len(rare_cats) > 0:
        print(f"   Rare categories (<1%): {rare_cats}")


=== Category Coverage & Dominance ===


--- MS Zoning ---
MS Zoning
RL         0.776
RM         0.158
FV         0.047
RH         0.009
C (all)    0.009
I (all)    0.001
A (agr)    0.001
Name: proportion, dtype: float64
   Rare categories (<1%): ['RH', 'C (all)', 'I (all)', 'A (agr)']

--- Street ---
Street
Pave    0.996
Grvl    0.004
Name: proportion, dtype: float64
!! Warning: 'Street' is dominated by category 'Pave' (99.6% of rows).
   Rare categories (<1%): ['Grvl']

--- Alley ---
Alley
None    0.932
Grvl    0.041
Pave    0.027
Name: proportion, dtype: float64
!! Warning: 'Alley' is dominated by category 'None' (93.2% of rows).

--- Lot Shape ---
Lot Shape
Reg    0.634
IR1    0.334
IR2    0.026
IR3    0.005
Name: proportion, dtype: float64
   Rare categories (<1%): ['IR3']

--- Land Contour ---
Land Contour
Lvl    0.899
HLS    0.041
Bnk    0.040
Low    0.020
Name: proportion, dtype: float64
!! Warning: 'Land Contour' is dominated by category 'Lvl' (89.9% of rows).

--- Utilities -

In [18]:
# === Step 5: Summary of Quality & Coverage ===
print("\n=== Summary of Quality & Coverage Assessment ===\n")

summary = {}

# 1. Low variability
threshold = 2
unique_counts = df_clean.nunique()
summary['low_variability'] = unique_counts[unique_counts <= threshold].index.tolist()

# 2. Dominant categories (>85%)
dominant_cols = []
rare_categories = {}

categorical_cols = df_clean.select_dtypes(include=["object", "string"]).columns
for col in categorical_cols:
    vc = df_clean[col].value_counts(normalize=True)
    
    if vc.iloc[0] > 0.85:
        dominant_cols.append((col, vc.index[0], float(vc.iloc[0])))

    rare = vc[vc < 0.01].index.tolist()
    if len(rare) > 0:
        rare_categories[col] = rare

# 3. Outliers
numeric_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns
outlier_summary = {}

for col in numeric_cols:
    q1, q3 = df_clean[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    outlier_summary[col] = int(((df_clean[col] < lower) | (df_clean[col] > upper)).sum())

outlier_summary = pd.Series(outlier_summary).sort_values(ascending=False)

# Print summary
print("• Low-variability columns:")
print(summary['low_variability'], "\n")

print("• Dominant-category columns (>85% one category):")
for col, cat, pct in dominant_cols:
    print(f"  - {col}: '{cat}' ({pct*100:.1f}%)")
print()

print("• Columns with rare categories (<1%):")
for col, cats in rare_categories.items():
    print(f"  - {col}: {cats}")
print()

print("• Top 10 variables with most outliers:")
print(outlier_summary.head(10))
print()

print("• Variables with zero outliers:")
print(outlier_summary[outlier_summary == 0].index.tolist())


=== Summary of Quality & Coverage Assessment ===

• Low-variability columns:
['Street', 'Central Air', 'Fence'] 

• Dominant-category columns (>85% one category):
  - Street: 'Pave' (99.6%)
  - Alley: 'None' (93.2%)
  - Land Contour: 'Lvl' (89.9%)
  - Utilities: 'AllPub' (99.9%)
  - Land Slope: 'Gtl' (95.2%)
  - Condition 1: 'Norm' (86.1%)
  - Condition 2: 'Norm' (99.0%)
  - Roof Matl: 'CompShg' (98.5%)
  - Heating: 'GasA' (98.5%)
  - Central Air: 'Y' (93.3%)
  - Electrical: 'SBrkr' (91.6%)
  - Paved Drive: 'Y' (90.5%)
  - Misc Feature: 'None' (96.4%)
  - Sale Type: 'WD' (86.6%)

• Columns with rare categories (<1%):
  - MS Zoning: ['RH', 'C (all)', 'I (all)', 'A (agr)']
  - Street: ['Grvl']
  - Lot Shape: ['IR3']
  - Utilities: ['NoSewr', 'NoSeWa']
  - Lot Config: ['FR3']
  - Land Slope: ['Sev']
  - Neighborhood: ['Blmngtn', 'Veenker', 'NPkVill', 'Blueste', 'Greens', 'GrnHill', 'Landmrk']
  - Condition 1: ['RRAe', 'PosA', 'RRNn', 'RRNe']
  - Condition 2: ['Feedr', 'Artery', 'PosA', '

In [19]:
# === Generate Markdown Summary Report ===

import os

report_path = "reports"
os.makedirs(report_path, exist_ok=True)

md_file = os.path.join(report_path, "quality_coverage_report.md")

# Collect data again in a clean structure
missing_pct = (df_clean.isna().sum() / len(df_clean) * 100).sort_values()

unique_counts = df_clean.nunique()
low_variability_cols = unique_counts[unique_counts <= 2].index.tolist()

categorical_cols = df_clean.select_dtypes(include=["object", "string"]).columns

dominant_cols = []
rare_categories = {}

for col in categorical_cols:
    vc = df_clean[col].value_counts(normalize=True)
    if not vc.empty and vc.iloc[0] > 0.85:
        dominant_cols.append((col, vc.index[0], float(vc.iloc[0])))

    rare = vc[vc < 0.01].index.tolist()
    if len(rare) > 0:
        rare_categories[col] = rare

numeric_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns

outlier_summary = {}
for col in numeric_cols:
    q1, q3 = df_clean[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    outlier_summary[col] = int(((df_clean[col] < lower) | (df_clean[col] > upper)).sum())

outlier_summary = pd.Series(outlier_summary).sort_values(ascending=False)


# === Write Markdown ===

with open(md_file, "w") as f:
    f.write("# Quality & Coverage Assessment Report\n\n")
    f.write("This document summarizes data quality checks performed on the cleaned Ames Housing dataset.\n\n")

    # Missingness
    f.write("## 1. Missingness per Column\n")
    f.write("All values should be 0% after cleaning.\n\n")
    f.write("```\n")
    f.write(missing_pct.to_string())
    f.write("\n```\n\n")

    # Low variability
    f.write("## 2. Low-Variability Columns (≤ 2 unique values)\n")
    f.write("```\n")
    f.write(str(low_variability_cols))
    f.write("\n```\n\n")

    # Dominant categories
    f.write("## 3. Dominant Categories (>85%)\n")
    if len(dominant_cols) == 0:
        f.write("No dominant categories detected.\n\n")
    else:
        f.write("| Column | Category | Percentage |\n")
        f.write("|--------|----------|------------|\n")
        for col, cat, pct in dominant_cols:
            f.write(f"| {col} | {cat} | {pct*100:.1f}% |\n")
        f.write("\n")

    # Rare categories
    f.write("## 4. Rare Categories (<1%)\n")
    if len(rare_categories) == 0:
        f.write("No rare categories detected.\n\n")
    else:
        for col, cats in rare_categories.items():
            f.write(f"- **{col}**: {cats}\n")
        f.write("\n")

    # Outliers
    f.write("## 5. Outlier Summary (IQR Method)\n")
    f.write("Top variables with most outliers:\n\n")
    f.write("```\n")
    f.write(outlier_summary.head(15).to_string())
    f.write("\n```\n\n")

    zero_outliers = outlier_summary[outlier_summary == 0].index.tolist()
    f.write("Variables with zero outliers:\n\n")
    f.write("```\n")
    f.write(str(zero_outliers))
    f.write("\n```\n")

print(f"Markdown report created: {md_file}")

Markdown report created: reports/quality_coverage_report.md
